In [5]:
%config IPCompleter.greedy=True
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))

In [6]:
import os
from datetime import datetime
from free_bwacs_model import FreeBWACS
%matplotlib inline

solutions_list = []
max_runs = 10
dct_with_header = False
save_results = True
_instance = 'CMT1'
_cluster_type = 'K-MEANS'

# Creación del archivo de texto plano con los resultados
now = datetime.now()
dt_string = now.strftime('%Y%m%d_%H-%M-%S')
aco_cluster = 'Free Ant + ' + _cluster_type  # Titulo del archivo .txt
# FA o RA (FREE ANT - RESTRICTED ANT) para el titulo del archivo
ant_type = 'FA'
# KMEN o KMED (K-MEANS - K-MEDOIDS) para el titulo del archivo
cluster_type = _cluster_type.replace('-', '').upper()[0:4]
instance = _instance
file_folder = 'instances_results/' + instance
file_folder_img = 'instances_results/' + instance + '/img_results'
if not os.path.exists(file_folder):
    os.makedirs(file_folder)
if not os.path.exists(file_folder_img):
    os.makedirs(file_folder_img)
file_name = ant_type + '_' + cluster_type + '_' + dt_string + '.txt'

for run in range(max_runs):
    aco_model = FreeBWACS(
        # Instance path + filename
        instance='CVRPLIB/CMT/' + _instance,
        # Max nodes selected
        max_nodes=9999,
        # Cluster type - options: | kmedoids - kmeans |
        cluster_type=_cluster_type.replace('-', '').lower(),
        # Metric distance - options: | euclidian - manhattan |
        metric='euclidian',
        # Tare for the heterogenoeus vehicles, this value is a percentage used by the next function: Vehicle Weight = Max Capacity * tare
        tare_percentage=0.15,
        # Max number of iterations
        max_iterations=100,
        # Define the total number of ants per iteration, through the next function: Total Ant = Total Nodes / total_ant_divider
        total_ant_divider=1,
        # Only accept solutions with the numbers of routes equal to K-Optimum - options: | True - False |
        only_k_optimum=True,
        # Place each ant in a different node of the candidate node list. Otherwise each ant will start its journey in the depot node.
        # options: | True - False |
        start_ant_on_best_nodes=True,
        # heuristic_type will let you choice what information gotta build the heuristic information matrix
        # 0 - Only distances data / Best beta: 5
        # 1 - Only energies data / Best gamma: 5
        # 2 - Distances data * energies data / Best parameters: 5 - 5
        # 3 - Only distances data of Savings algorithm / Best delta: 1
        # 4 - Only energies data of Savings algorithm / Best eta: 2
        # 5 - Distances data * distances savings data / Best parameters: 4 - 5
        # 6 - Energies data * energies savings data / Best parameters: 5 - 5
        # 7 - Distances data * energies data * distances savings data / Best parameters: 3 - 4 - 3
        # 8 - Distances data * energies data * energies savings data / Best parameters: 3 - 4 - 3
        # 9 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
        # 10 - Distances data * energies data * distances savings data * capacity utilization data / Best parameters: 2 - 4 - 4 - 2
        # 11 - Distances data * energies data * distances savings data * energies savings data * capacity utilization data / Best parameters: 2 - 4 - 3 - 4 - 1
        heuristic_type=8,
        # Importance of pheromones trace. Recommended alpha value in: [1, 4]
        alpha=3,
        # Importance of distance heuristic information. Recommended beta value in: [2, 5]
        beta=2,
        # Importance of energy heuristic information. Recommended gamma value in: [2, 5]
        gamma=3,
        # Importance of distances's Savings Algorithm information. Recommended delta value in: [0, 2]
        delta=1,
        # Importance of energies's Savings Algorithm information. Recommended eta value in: [0, 2]
        eta=1,
        # Importance of vehicle's capacity utilization. Recommended mi value in: [0, 2]
        mi=0,
        # Define the strategy used for pheromone updating. - options: | 0 - 1 |
        # The value 0 is for the classic function in ACO: 1 / Best_Solution_Quality
        # The value 1 if for the improve function know as Ant Weight Strategy.
        pheromone_updating_strategy=0,
        # Determine if every single ant can update the pheromones whit - options: | True - False |
        local_ant_update_pheromones=False,
        # Determine if the best ant of current iteration can update the pheromones - options: | True - False |
        best_iteration_ant_update_pheromones=False,
        # Determine if the best global ant can update the pheromones - options: | True - False |
        best_global_ant_update_pheromones=True,
        # Defines if the worst solution of the current iteration should penalize the pheromone arcs. - options: | True - False |
        penalize_worst_solution=True,
        # Want to mutate the pheromones matrix? - options: | True - False |
        mutate_pheromones_matrix=True,
        # Evaporation rate of the pheromones by the next function: phoromone =  (1 - p) * pheromone. Recommended value for p: 0.02.
        p=0.02,
        # Mutation probability for every row of the pheromones matrix, such as: mutate if (random number between 0 and 1) <= Pm
        # Recommended value for Pm: 0.3.
        Pm=0.3,
        # Mutation intensity for the pheromones matrix. Recommended value for sigma in: [2, 4]
        sigma=4,
        # Constant used in Pseudorandom Transition Rule. This is a probability of choice the next node by argmax, define in Ant Colony System.
        # Recommended value for l0 in: [0.2, 0.4]
        l0=0.3,
        # Constant used for a lot of functions, such as: f(.) = H / specific_values.
        H=1,
        # Want to do a Local Search to the solution of every single ant? - options: | True - False |
        ls_ant_solution=False,
        # Want to do a Local Search to the best iteration solution? - options: | True - False |
        ls_best_iteration=False,
        # Want to do a Local Search to the best global solution? - options: | True - False |
        ls_best_global=False,
        # Want to use the Normalized Matrix? - options: | True - False |
        use_normalized_matrix=False,
        # Use Matplotlib to draw a 2-D graph with the nodes of the problem. - option: | True - False |
        print_instance=False,
        # Use Matplotlib to draw the solution of clustering process. - option: | True - False |
        print_clusters=False,
        # Use Matplotlib to draw the final solution of the problem. - option: | True - False |
        print_solution=False,
        # Show the distance matrix in a html table. - option: | True - False |
        print_distance_matrix=False,
        # Show the energy matrix in a html table. - option: | True - False |
        print_energy_matrix=False,
        # Show the distance savings matrix in a html table. - option: | True - False |
        print_distance_saving_matrix=False,
        # Show the energy savings matrix in a html table. - option: | True - False |
        print_energy_saving_matrix=False,
        # Show the final heuristic matrix in a html table. - option: | True - False |
        print_combination_matrix=False,
        # Show the pheromone matrix in a html table. - option: | True - False |
        print_pheromone_matrix=False,
        # Output folder/name solution draw
        output_sol_img=file_folder_img + '/' + \
        str(run + 1) + '__' + ant_type + '_' + \
        cluster_type + '_' + dt_string + '.png'
    )

    solution_energy, solution_distance, solution_time = aco_model.solve()
    solutions_list.append((solution_energy, solution_distance, solution_time))

print(solutions_list)
print("Best Solution: " + str(min(solutions_list, key=lambda x: x[0])) + " on run: " + str(solutions_list.index(min(solutions_list, key=lambda x: x[0])) + 1))

if (save_results):
    file = open(file_folder + '/' + file_name, 'a')
    if dct_with_header:
        file.write('Resultados para la ' + instance + ' (' +
                aco_cluster + ') [energia, distancia, tiempo de ejecución]\n\n')
    for i, solution in enumerate(solutions_list):
        if dct_with_header:
            file.write(str(i + 1) + '| ' + str(solution) + '\n')
        else:
            str_sol = str(solution).replace('(', '').replace(')',
                                                            '').replace(',', '').replace('.', ',')
            file.write(str_sol + '\n')

    file.close()


• INITIALIZING ALGORITHM •
------------------------------

• 0. Reading instance file
------------------------------

    - Parameters:
        > Instance name: CVRPLIB/CMT/CMT1
        > Number of nodes: 51
        > Nodes and demands: {0: 0, 1: 7, 2: 30, 3: 16, 4: 9, 5: 21, 6: 15, 7: 19, 8: 23, 9: 11, 10: 5, 11: 19, 12: 29, 13: 23, 14: 21, 15: 10, 16: 15, 17: 3, 18: 41, 19: 9, 20: 28, 21: 8, 22: 8, 23: 16, 24: 10, 25: 28, 26: 7, 27: 15, 28: 14, 29: 6, 30: 19, 31: 11, 32: 12, 33: 23, 34: 26, 35: 17, 36: 6, 37: 9, 38: 15, 39: 14, 40: 7, 41: 27, 42: 13, 43: 11, 44: 16, 45: 10, 46: 5, 47: 25, 48: 17, 49: 18, 50: 10}
        > Total demand: 777 units
        > Vehicles capacity: 160 units
        > K-Optimum: 5
        > Tightness ratio: 0.97125
        > Total of iterations: 100
        > Number of ants per iteration: 50

• 1. Starting clustering process
------------------------------

    > CLUSTER TYPE: KMEANS
    > ITERACIÓN 1
        - Centroides actuales: [array([57, 58]), array([17

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:401: RuntimeWarning: divide by zero encountered in true_divide
  self.distances_matrix_normalized = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:418: RuntimeWarning: divide by zero encountered in true_divide
  self.energies_matrix_normalized = self.np.divide(1, self.energies_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:546: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.power(self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:547: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.power(self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:550: RuntimeWarning: inva

        - Nodos sin asignar: [18 39], [9 7]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 8682.729335656946

    > ITERACIÓN 38
        - Centroides actuales: [array([48.90909091, 58.36363636]), array([50.8, 35.6]), array([5., 6.]), array([38.5, 18.1]), array([19.2, 55.8])]
        - Nodos sin asignar: [38], [14]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9938.838033288544

    > ITERACIÓN 39
        - Centroides actuales: [array([48.90909091, 58.36363636]), array([50.8, 35.6]), array([12.25, 21.  ]), array([59., 15.]), array([19.2, 55.8])]
        - Nodos sin asignar: [24 42], [28 11]
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 8844.914022470824

    > ITERACIÓN 40
        - Centroides actuales: [array([48.36363636, 57.72727273]), array([46.6, 36.5]), array([ 7., 38.]), array([45.77777778, 18.77777778]), array([ 5., 64.])]
        - Nodos sin asignar: [40], [27]
        - Mejor c

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:401: RuntimeWarning: divide by zero encountered in true_divide
  self.distances_matrix_normalized = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:418: RuntimeWarning: divide by zero encountered in true_divide
  self.energies_matrix_normalized = self.np.divide(1, self.energies_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:546: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.power(self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:547: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.power(self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:550: RuntimeWarning: inva

        - Nodos sin asignar: [12], [23]
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 9205.552639738571

    > ITERACIÓN 8
        - Centroides actuales: [array([43.8, 61.5]), array([47.        , 39.45454545]), array([ 5., 25.]), array([50.11111111, 23.66666667]), array([15.66666667, 51.77777778])]
        - Nodos sin asignar: [11], [29]
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 9274.450376400218

    > ITERACIÓN 9
        - Centroides actuales: [array([43.72727273, 58.54545455]), array([31., 32.]), array([12.25, 21.  ]), array([45.18181818, 21.36363636]), array([17.66666667, 51.11111111])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 10115.12837053417

    > ITERACIÓN 10
        - Centroides actuales: [array([44.4, 57.2]), array([40.09090909, 35.54545455]), array([12.25, 21.  ]), array([49. , 22.3]), array([25.36363636, 54.18181818])]
        -

...

        - Centroides actuales: [array([43.8, 61.5]), array([47.        , 39.45454545]), array([ 5., 25.]), array([50.11111111, 23.66666667]), array([15.66666667, 51.77777778])]
        - Nodos sin asignar: [11], [29]
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 9274.450376400218

    > ITERACIÓN 9
        - Centroides actuales: [array([43.72727273, 58.54545455]), array([31., 32.]), array([12.25, 21.  ]), array([45.18181818, 21.36363636]), array([17.66666667, 51.11111111])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 10115.12837053417

    > ITERACIÓN 10
        - Centroides actuales: [array([44.4, 57.2]), array([40.09090909, 35.54545455]), array([12.25, 21.  ]), array([49. , 22.3]), array([25.36363636, 54.18181818])]
        - Nodos sin asignar: [43], [16]
        - Mejor costo anterior: 9131.213913563224
        - Costo total actual: 8770.75270035537

    > ITERACIÓN 11
        -

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:401: RuntimeWarning: divide by zero encountered in true_divide
  self.distances_matrix_normalized = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:418: RuntimeWarning: divide by zero encountered in true_divide
  self.energies_matrix_normalized = self.np.divide(1, self.energies_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:546: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.power(self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:547: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.power(self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:550: RuntimeWarning: inva

        - Centroides actuales: [array([37., 69.]), array([48.6, 34.4]), array([19.88888889, 25.66666667]), array([35.8, 20.2]), array([19.2, 55.8])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8066.528567941032
        - Costo total actual: 9867.190655216931

    > Mejor Clusterización: [array([ 2, 16, 29, 11, 22, 32, 20,  1,  3, 35], dtype=int64), array([ 9, 49, 30, 50, 10, 38, 34, 21, 39, 33, 45], dtype=int64), array([18, 13,  4, 41, 25, 14, 19], dtype=int64), array([17, 37, 12, 47, 44,  5, 15, 46, 42, 40], dtype=int64), array([48,  7, 23,  8, 26,  6, 27, 31, 24, 43], dtype=int64)], con costo final: 8066.528567941032, nodos sin asignar: [28 36]
    > Mejor Clusterización (Contraint): [array([ 2, 29, 20, 16, 21, 22,  3, 11, 35, 36], dtype=int64), array([49,  9, 38,  5, 10, 30, 50, 34, 39, 43], dtype=int64), array([18, 47, 12, 14, 46, 25, 24], dtype=int64), array([ 4, 17, 37, 44, 42, 41, 19, 15, 13, 45, 40, 33], dtype=int64), array([48,  8, 27, 23,  6,  7,  1, 2

e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:401: RuntimeWarning: divide by zero encountered in true_divide
  self.distances_matrix_normalized = self.np.divide(1, self.distances_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:418: RuntimeWarning: divide by zero encountered in true_divide
  self.energies_matrix_normalized = self.np.divide(1, self.energies_matrix)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:546: RuntimeWarning: divide by zero encountered in true_divide
  _distance_matrix = self.np.power(self.np.divide(1, self.distances_matrix), self.beta)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:547: RuntimeWarning: divide by zero encountered in true_divide
  _energy_matrix = self.np.power(self.np.divide(1, self.energies_matrix), self.gamma)
e:\0 - PROYECTOS\1 - PERSONALES\BWACS-EMVRP\improved-bwacs\free_bwacs_model.py:550: RuntimeWarning: inva

        - Centroides actuales: [array([49.54545455, 51.27272727]), array([46., 10.]), array([ 5., 25.]), array([25.33333333, 16.88888889]), array([25.6, 57. ])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8306.319175559167
        - Costo total actual: 11012.655751743518

    > ITERACIÓN 28
        - Centroides actuales: [array([49.54545455, 48.63636364]), array([49. , 22.3]), array([ 9.85714286, 24.85714286]), array([35.27272727, 33.27272727]), array([23.63636364, 59.27272727])]
        - Nodos sin asignar: [], []
        - Mejor costo anterior: 8306.319175559167
        - Costo total actual: 9547.94009204571

    > ITERACIÓN 29
        - Centroides actuales: [array([49.54545455, 48.63636364]), array([52.27272727, 29.54545455]), array([12.        , 27.71428571]), array([28.90909091, 26.54545455]), array([23. , 60.4])]
        - Nodos sin asignar: [27 35], [14  6]
        - Mejor costo anterior: 8306.319175559167
        - Costo total actual: 8066.528567941032

